In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos > 25]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,0.71,4.2,1,0.00,2.5,1,1.00,7.0,0,28
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,1,31
7,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,0.00,0.0,1,0.00,0.4,0,0.00,0.0,0,32
11,36894,ESPORTSTMNT02_2577994,0,20,2,1,5,3,6,20,...,0.00,0.0,1,1.00,12.0,13,0.46,1.9,0,30
13,36896,ESPORTSTMNT02_2576974,0,20,2,1,4,21,20,3,...,0.50,3.3,9,0.44,4.3,8,0.75,2.1,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,42932,ESPORTSTMNT03_2920219,19,20,2,20,24,3,20,4,...,0.50,2.6,2,0.50,6.5,1,0.00,3.7,1,30
3994,40959,ESPORTSTMNT03_2785772,1,2,19,5,20,22,2,20,...,0.00,1.6,19,0.84,6.5,35,0.60,3.5,1,30
3996,40957,ESPORTSTMNT03_2785705,1,19,21,20,2,23,5,1,...,0.00,1.4,1,0.00,1.5,9,0.44,3.2,0,38
3997,43349,ESPORTSTMNT01_3082925,19,2,20,1,21,5,22,24,...,0.50,4.7,6,0.33,2.8,0,0.00,0.0,0,31


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
0,19,1,2,1,3,24,2,23,6,1,...,14,0.71,4.2,1,0.00,2.5,1,1.00,7.0,28
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
7,19,1,2,5,1,20,21,4,1,3,...,0,0.00,0.0,1,0.00,0.4,0,0.00,0.0,32
11,0,20,2,1,5,3,6,20,2,4,...,0,0.00,0.0,1,1.00,12.0,13,0.46,1.9,30
13,0,20,2,1,4,21,20,3,5,22,...,2,0.50,3.3,9,0.44,4.3,8,0.75,2.1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,19,20,2,20,24,3,20,4,21,23,...,2,0.50,2.6,2,0.50,6.5,1,0.00,3.7,30
3994,1,2,19,5,20,22,2,20,24,4,...,1,0.00,1.6,19,0.84,6.5,35,0.60,3.5,30
3996,1,19,21,20,2,23,5,1,24,3,...,2,0.00,1.4,1,0.00,1.5,9,0.44,3.2,38
3997,19,2,20,1,21,5,22,24,3,1,...,2,0.50,4.7,6,0.33,2.8,0,0.00,0.0,31


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.932177,0.929134,0.944,0.936508,0.932177
1,Support Vector Machine (Linear Kernel),0.919327,0.914062,0.936,0.924901,0.919327
2,Support Vector Machine (RBF Kernel),0.901628,0.886364,0.936,0.910506,0.901628
3,Decission Tree,0.880354,0.864662,0.920,0.891473,0.880354
4,Adaboost,0.902478,0.898438,0.920,0.909091,0.902478
5,Random Forest,0.910903,0.906250,0.928,0.916996,0.910903
6,Gradient Boosting Classifier,0.915327,0.913386,0.928,0.920635,0.915327
